<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E7%8E%89%E5%B1%B12021%E5%86%AC%E5%AD%A3%E8%B3%BD%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!python -m pip install "dask[complete]" 

In [12]:
%%time
!pip install gdown

CPU times: user 28.8 ms, sys: 102 ms, total: 131 ms
Wall time: 3.2 s


In [3]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd


In [4]:
import tensorflow as tf


# 資料io前處理

In [7]:
'''
!cp /content/drive/MyDrive/02_GNL_DATA/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz /content
!gunzip /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz
!tar -xvf /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz


tbrain_cc_training_48tags_hash_final.csv


In [9]:
%%time


'''
df = dd.read_csv('/content/tbrain_cc_training_48tags_hash_final.csv',assume_missing=True)

## 以chid, dt排序 
df = df.set_index('chid')
df = df.map_partitions(lambda df: df.sort_values(['chid', 'dt']))

## 把csv切成32個parquet格式
df = df.repartition(npartitions=32) 
df = dd.to_parquet(df,path='/content/data.parquet', engine='pyarrow')

## 打包壓縮
!zip -r esun2021winterdata_parquet.zip /content/data.parquet
'''


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 23.6 µs


"\ndf = dd.read_csv('/content/tbrain_cc_training_48tags_hash_final.csv',assume_missing=True)\n\n## 以chid, dt排序 \ndf = df.set_index('chid')\ndf = df.map_partitions(lambda df: df.sort_values(['chid', 'dt']))\n\n## 把csv切成32個parquet格式\ndf = df.repartition(npartitions=32) \ndf = dd.to_parquet(df,path='/content/data.parquet', engine='pyarrow')\n\n## 打包壓縮\n!zip -r esun2021winterdata_parquet.zip /content/data.parquet\n"

In [10]:
#!cp /content/esun2021winterdata_parquet.zip /content/drive/MyDrive/03_RESOURCE

# 讀入乾淨資料EDA

In [14]:
!gdown https://drive.google.com/uc?id=1pyWFfdT9r-oMoeTmB4NqCTQ0IS7tzzte
!unzip /content/esun2021winterdata_parquet.zip

Downloading...
From: https://drive.google.com/uc?id=1pyWFfdT9r-oMoeTmB4NqCTQ0IS7tzzte
To: /content/esun2021winterdata_parquet.zip
100% 203M/203M [00:00<00:00, 234MB/s]


In [16]:
## 直接讀取打包好排序過的資料
## 載入切好格式

org_df = dd.read_parquet('/content/data.parquet', engine='pyarrow')


In [18]:
%%time
##看看一個id都幾筆資料
org_df.groupby('chid').size().describe().compute()

CPU times: user 38.6 s, sys: 28.1 s, total: 1min 6s
Wall time: 21.1 s


count    500000.000000
mean         65.951306
std          55.929864
min           1.000000
25%          24.000000
50%          50.000000
75%          93.000000
max         441.000000
dtype: float64

In [40]:
%%time
org_df['dt'].unique().compute()

CPU times: user 1.87 s, sys: 384 ms, total: 2.25 s
Wall time: 771 ms


0      1.0
1      2.0
2      3.0
3      4.0
4      5.0
5      6.0
6      7.0
7      8.0
8      9.0
9     10.0
10    11.0
11    12.0
12    13.0
13    14.0
14    15.0
15    16.0
16    17.0
17    18.0
18    19.0
19    20.0
20    21.0
21    22.0
22    23.0
23    24.0
Name: dt, dtype: float64

# 資料整併

In [29]:
## 消費類別在這些list才是預測範圍

pred_list = ['2',
'6',
'10',
'12',
'13',
'15',
'18',
'19',
'21',
'22',
'25',
'26',
'36',
'37',
'39',
'48']

In [26]:

## 基本資料取最新一筆
profile_df = org_df.groupby('chid').last().compute()
profile_df = profile_df[
  [
  'masts', 'educd', 'trdtp', 'naty', 'poscd', 
  'cuorg','gender_code', 'age'
  ]
]


In [32]:
## 前11個月預測第12月作為訓練
train_org = org_df[org_df['dt']<11]
train_y_org = org_df[org_df['dt']==11]

In [33]:
%%time
train_y_org = train_y_org.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

## 只留下要預測的類別
train_y_org= train_y_org[pred_list]


CPU times: user 39.4 s, sys: 23.5 s, total: 1min 2s
Wall time: 22.1 s


In [34]:
train_y_org.head()

shop_tag,2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48
chid,,,,,,,,,,,,,,,,
10000000.0,0.00000,0.0,10512.141355,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
10000001.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,1332.986072,0.000000,0.0,0.000000
10000004.0,0.00000,0.0,0.000000,0.000000,0.0,5773.036485,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,5419.225995
10000006.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,4268.476774,0.0,0.000000
10000008.0,10482.02793,0.0,0.000000,6735.554834,0.0,0.000000,4953.135169,15007.05093,36527.183292,0.0,0.0,0.0,0.000000,44217.947863,0.0,0.000000


In [35]:
## by id 將絕對金額轉換為各類佔比 (加總為1) 
train_y = train_y_org.div(train_y_org.sum(axis=1), axis=0).fillna(0)

In [36]:
train_y

shop_tag,2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48
chid,,,,,,,,,,,,,,,,
10000000.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10000001.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000
10000004.0,0.000000,0.0,0.000000,0.000000,0.000000,0.515806,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.484194
10000006.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000
10000008.0,0.088889,0.0,0.000000,0.057118,0.000000,0.000000,0.042003,0.127262,0.309755,0.0,0.0,0.0,0.0,0.374973,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499992.0,0.000000,0.0,0.000000,0.000000,0.804538,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.195462
10499993.0,0.000000,0.0,0.499181,0.168151,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.332668,0.000000,0.000000
10499996.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [37]:
## 將過去11個月個類別的消費金額加總
train_df = train_org.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

## 合併基本資料
train_df = train_df.merge(profile_df, how='left', on='chid')


In [38]:
## 取y跟x皆有的交集
train_int_id = list(set(train_df.index)&set(train_y_org.index))
train_int_id.sort()

In [ ]:
# train_df[train_int_id], train_y[train_int_id], 就是訓練集了

# Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

model.compile(optimizer='rmsprop',
       loss='mae')



model.fit(train_df.loc[train_int_id].values, train_y.loc[train_int_id].values, batch_size=1, epochs=100)


# Naive baseline

In [19]:
%%time

## 計算by id各類別總計消費金額
naive_pred_df = org_df.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

CPU times: user 52.4 s, sys: 25.1 s, total: 1min 17s
Wall time: 29.1 s


In [20]:
naive_pred_df.head()

shop_tag,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,5,6,7,8,9,other
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,0.0,191770.225706,0.0,76268.969509,0.0,0.0,4684.483372,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,4235.791755,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,62218.660703,81337.066461,0.0,9156.086033,0.0,0.00000,0.0,0.0,0.000000,40746.482820
10000001.0,0.0,12847.384866,0.0,44104.144043,0.0,0.0,57821.590353,12643.832959,0.000000,8019.906666,0.0,17649.573492,0.0,25146.943067,3945.661088,0.0,0.0,0.0,39478.512075,0.0,0.0,0.0,6308.563642,0.0,595.018412,0.0,0.0,51329.246565,0.0,26998.210065,56105.751640,0.0,49331.267298,0.0,18721.636466,0.0,14317.563657,0.0,6048.351042,63374.858212,0.000000,0.0,84246.368876,0.0,26023.22562,0.0,0.0,0.000000,50488.365846
10000002.0,0.0,16755.695323,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,3571.095195,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5692.923612,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,37649.183244
10000003.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,8554.030804,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8023.582985,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000
10000004.0,0.0,0.000000,0.0,15271.878083,0.0,0.0,5773.036485,0.000000,6037.642859,0.000000,0.0,10825.822388,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,12617.127283,0.0,0.0,0.000000,0.0,11546.072969,34238.290641,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,19790.314068,0.000000,0.0,5419.225995,0.0,0.00000,0.0,0.0,10659.532107,12386.980405


In [22]:

# 取最大 
# np.argmax(newdf,axis=1)


## 取topn最大值的位置 (為了直接映射colname, 而colname即是類別)

## n=0~2
n=0
(-naive_pred_df.values).argsort(axis=1)[:,n]

array([ 1, 42, 48, ..., 35, 39, 14])

In [ ]:
# 直接把位子映射colname,colname即是類別
top1 = [list(naive_pred_df.columns)[x] for x in (-naive_pred_df.values).argsort(axis=1)[:,0]]
top2 = [list(naive_pred_df.columns)[x] for x in (-naive_pred_df.values).argsort(axis=1)[:,1]]
top3 = [list(naive_pred_df.columns)[x] for x in (-naive_pred_df.values).argsort(axis=1)[:,2]]

In [ ]:
top_df = pd.DataFrame({'chid':newdf.index, 'top1':top1, 'top2':top2, 'top3':top3})
top_df['chid'] = top_df['chid'].astype(int)

In [ ]:
## 不再預測範圍的排掉
## 37 15 36為先驗top3最多的消費類別

top_df['top1'] = np.where(top_df['top1'].isin(pred_list), top_df['top1'], np.nan)
top_df['top2'] = np.where(top_df['top2'].isin(pred_list), top_df['top2'], np.nan)
top_df['top3'] = np.where(top_df['top3'].isin(pred_list), top_df['top3'], np.nan)

In [ ]:
top_df = top_df.fillna('37')
top_df.to_csv('submit2.csv', index=False)